# Bicicletas Londres

## Importacion

In [59]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import csv
from itertools import islice
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
from sklearn.model_selection import train_test_split

## Obtención de csv

In [2]:
csvfiler = r"C:\Users\David\PycharmProjects\first_project\Bicicletas\LondonBikeJourneyAug2023.csv"

Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration

### División de los datos deseados

In [3]:
start_station_categories = []
end_station_categories = []
total_duration_ms = []
model_bike = []

In [4]:
with open (csvfiler, newline="") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        start_station_categories.append(row["Start station"])
        end_station_categories.append(row["End station"])
        total_duration_ms.append(row["Total duration (ms)"])
        model_bike.append(row["Bike model"])

### Creación de listas de los diferentes nombres de estaciones de start con la cantidad de usos de cada una

In [58]:
label_encoder = LabelEncoder()
lista_start_estaciones = list(Counter(start_station_categories).keys())
start_counter = Counter(start_station_categories)
codigo_start = label_encoder.fit(lista_start_estaciones)
codigo_start = label_encoder.transform(lista_start_estaciones)

### Creación de listas de los diferentes nombres de estaciones de end con la cantidad de usos de cada una

In [6]:
label_encoder = LabelEncoder()
end_counter = Counter(end_station_categories)
lista_end_estaciones = list(Counter(end_station_categories).keys())
codigo_end = label_encoder.fit(lista_end_estaciones)
codigo_end = label_encoder.transform(lista_end_estaciones)

### Creación de listas de los diferentes nombres de bicicletas con la cantidad de usos de cada una

In [7]:
label_encoder = LabelEncoder()
bike_counter = Counter(model_bike)
print(bike_counter)
lista_bikes = list(Counter(model_bike).keys())
codigo_bike = label_encoder.fit(lista_bikes)
codigo_bike = label_encoder.transform(lista_bikes)

Counter({'CLASSIC': 716639, 'PBSC_EBIKE': 59888})


## Iniciación geolocalizador

In [183]:
# Función para obtener las coordenadas geográficas de una estación
geolocalizador = Nominatim(user_agent="registros_bicicletas_londres")
def obtener_coordenadas(nombre_estacion):
    location = geolocalizador.geocode(nombre_estacion)
    return (location.latitude, location.longitude) if location else None

# Calcular la distancia entre cada par de estaciones
def calcular_distancia(coord1, coord2):
    if coord1 and coord2:
        return geodesic(coord1, coord2).kilometers
    else:
        return None

## Creación de gráficos de uso start

In [209]:
lista_lugares = ["Abadía de Westminster", "Big Ben", "London Eye", "Tower Bridge", "Hyde Park", "National Gallery", "Palacio de Buckingham", "Covent Garden"]
lista_lugares_coordenadas = list(map(obtener_coordenadas, lista_lugares))

In [189]:
print(calcular_distancia(lista_lugares_coordenadas[0], lista_lugares_coordenadas[6]))

1.0959117836406396


In [210]:
def lugar_mas_cercano(estacion, lugares):
    if estacion and lugares:
        lista_cercania = []
        for i in lugares:
            lista_cercania.append(calcular_distancia(obtener_coordenadas(estacion), i))
        return (min(lista_cercania), lista_cercania.index(min(lista_cercania)))
    else:
        return None
datos = lugar_mas_cercano(list(start_counter.keys())[3], lista_lugares_coordenadas)

In [ ]:
# Define los valores para el gráfico polar
# Define los límites para cada sección
theta_limits = [0, np.pi/4, np.pi/2, 3*np.pi/4, np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4, 2*np.pi]
N = len(lista_start_estaciones)
grupo_perteneciente = []
for i in lista_start_estaciones:
    try:
        grupo_perteneciente.append(lugar_mas_cercano(i, lista_lugares_coordenadas))
    except:
        pass
r = 2 * np.array(list(start_counter.values()))
theta = tetha_limits[grupo_perteneciente[1]]
area = 200 * r**2

# Define los colores basados en los ángulos
colors = theta

# Crea el gráfico polar
fig = plt.figure()
ax = fig.add_subplot(projection='polar')



# Define los radios para cada sección
r_limits = [0.1, 0.3, 0.5, 0.7, 0.9]

# Itera sobre cada punto y ajusta su posición relativa dentro de cada sección
for i in range(N):
    # Encuentra la sección correspondiente
    section = 0
    while theta[i] > theta_limits[section]:
        section += 1
    section -= 1

    # Ajusta el radio para que esté dentro de la sección
    r[i] = r_limits[section] + (r_limits[section + 1] - r_limits[section]) * (r[i] - r.min()) / (r.max() - r.min())

# Grafica los puntos ajustados
c = ax.scatter(theta, r, c=colors, s=area, cmap='hsv', alpha=0.75)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(300, 6), layout='constrained')
ax.bar(keys, list(values))

## **Conversion de datos a numéricos**

In [45]:
lista_buena_start = []
contador = 0
for i in start_counter.values():
    for x in range(i):
        lista_buena_start.append(codigo_start[contador])
    contador += 1
        

In [46]:
lista_buena_end = []
contador = 0
for i in end_counter.values():
    for x in range(i):
        lista_buena_end.append(codigo_end[contador])
    contador += 1

In [47]:
lista_buena_bike = []
contador = 0
for i in bike_counter.values():
    for x in range(i):
        lista_buena_bike.append(codigo_bike[contador])
    contador += 1

In [50]:
print(len(lista_buena_start), len(lista_buena_end), len(lista_buena_bike))

776527 776527 776527


In [51]:
np_lista_buena_start = np.array(lista_buena_start)
np_lista_buena_end = np.array(lista_buena_end)
np_lista_buena_bike = np.array(lista_buena_bike)

In [52]:
print(np_lista_buena_start.shape, np_lista_buena_end.shape, np_lista_buena_bike.shape)

(776527,) (776527,) (776527,)


In [168]:
X = np.column_stack((np_lista_buena_start, np_lista_buena_end, np_lista_buena_bike))

In [136]:
X = X

In [169]:
total_duration_ms = np.array(total_duration_ms, dtype="float32")
total_duration_ms= total_duration_ms / 1000

In [170]:
x_train, x_test, y_train, y_test = train_test_split(X, total_duration_ms, test_size=0.2, random_state=42)

In [171]:
x_train = np.array(x_train, dtype="float32")
x_test = np.array(x_test, dtype="float32")
print (type(x_train))
print(type(y_train))
print(x_test.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(155306, 3)


In [172]:
model_duration = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape= (3,)),
    tf.keras.layers.Dense(32, activation = "relu"),
    tf.keras.layers.Dense(64, activation = "relu"),
    tf.keras.layers.Dense(128, activation = "relu"),
    tf.keras.layers.Dense(1)
])

In [173]:
model_duration.compile(
    optimizer = "adam",
    loss='mean_squared_error',
    metrics=['mean_squared_error', 'mean_absolute_error']
)

In [174]:
print(len(total_duration_ms))

776527


In [175]:
model_duration.fit(x_train, y_train, verbose=1, epochs= 10)

Epoch 1/10
19414/19414 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 1.7578 - mean_absolute_error: 0.1956 - mean_squared_error: 1.7578
Epoch 2/10
19414/19414 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 4.3404e-05 - mean_absolute_error: 0.0028 - mean_squared_error: 4.3404e-05
Epoch 3/10
19414/19414 ━━━━━━━━━━━━━━━━━━━━ 32s 2ms/step - loss: 1.0525e-30 - mean_absolute_error: 1.1624e-16 - mean_squared_error: 1.0525e-30
Epoch 4/10
19414/19414 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 7.1349e-08 - mean_absolute_error: 1.5359e-05 - mean_squared_error: 7.1349e-08
Epoch 5/10
19414/19414 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 1.1974e-30 - mean_absolute_error: 1.1783e-16 - mean_squared_error: 1.1974e-30
Epoch 6/10
19414/19414 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 1.0357e-30 - mean_absolute_error: 1.1551e-16 - mean_squared_error: 1.0357e-30
Epoch 7/10
19414/19414 ━━━━━━━━━━━━━━━━━━━━ 31s 2ms/step - loss: 1.0738e-30 - mean_absolute_error: 1.1670e-16 - mean_squared_error: 1.0738e-30
Epoch 8/10
1941

In [176]:
prediccion = model_duration.predict(x_test)
model_duration.evaluate(x_test, y_test)

4854/4854 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step
4854/4854 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - loss: 1.8567e-30 - mean_absolute_error: 1.2088e-16 - mean_squared_error: 1.8567e-30


[3.765697093254679e-30, 3.765697093254679e-30, 1.2668628423274242e-16]

In [178]:
print(prediccion[0:20])
print(y_test[0:20])

[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
[1.60685955e-17 6.62996974e-17 2.07212201e-17 1.34940276e-16
 1.99591906e-16 1.17645068e-16 1.01393516e-16 2.40915129e-17
 9.18736867e-17 9.15912872e-17 2.77238417e-17 9.76916715e-17
 1.65333023e-16 2.47946760e-17 3.60797059e-17 8.41858039e-17
 4.93203694e-17 8.05871116e-17 4.74037820e-17 1.25504713e-16]
